In [1]:
import glob         #regex for reading files
import re           #regex
import multiprocessing
import os               #deals with os(for reading files etc)

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

from gensim.models import Word2Vec

import json
from pprint import pprint

In [16]:
filenames = glob.glob("../Datasets/re3d-defense/re3d/*/documents.json")
filenames

['../Datasets/re3d-defense/re3d/Wikipedia/documents.json',
 '../Datasets/re3d-defense/re3d/BBC Online/documents.json',
 '../Datasets/re3d-defense/re3d/Australian Department of Foreign Affairs/documents.json',
 '../Datasets/re3d-defense/re3d/UK Government/documents.json',
 '../Datasets/re3d-defense/re3d/US State Department/documents.json',
 '../Datasets/re3d-defense/re3d/CENTCOM/documents.json',
 '../Datasets/re3d-defense/re3d/Delegation of the European Union to Syria/documents.json']

In [17]:
#Stores the json-data into the variable data

data = []
for fn in filenames:
    with open(fn) as f:
        for line in f:
            data.append(json.loads(line))

In [18]:
print(len(data))

98


In [19]:
raw_text = u""
for i in range( len(data) ):
    raw_text += data[1]['text']

In [20]:
print(raw_text)

The Brandbergen Mosque, officially the Islamic Association in Brandbergen is a mosque located in Brandbergen, Haninge Municipality, south of Stockholm, Sweden.
"In 2004 an Arabic-language manual, which carried the logo and address of the Brandbergen Mosque, was spread on the internet. The manual described the construction of simple chemical weapons, including how to build a chemical munition from an ordinary artillery round.[1] On December 7, 2006, the Swedish citizen Mohamed Moumou, who is described by the United States Department of the Treasury as an ""uncontested leader of an extremist group centered around the Brandbergen Mosque in Stockholm"", was put on the United Nations Security Council Committee 1267 list of foreign terrorists.[2]"
In November 2006 the Islamic Association got permission to build a Muslim Cultural Center next to the city center. The cultural center will include an assembly hall, library, restaurant, gyms and baths.[3] It is meant to serve as a meeting place fo

In [21]:
# tokenizer = nltk.data.load("/Users/Abhishek/nltk_data/tokenizers/punkt/english.pickle")
raw_sentences = sent_tokenize(raw_text)

stop_words = set(stopwords.words("english"))

def filter_and_tokenize(raw_sent):
    filtered_words = []
    
    clean = re.sub("[^a-zA-Z]", " ", raw_sent)  #remove all non-letters
    words = clean.split()
    
    for w in words:
        if w not in stop_words:
            filtered_words.append(w)
            
    return filtered_words

filtered_sentences = []
for sent in raw_sentences:
    if len(sent) > 0:
        filtered_sentences.append(filter_and_tokenize(sent))

In [22]:
print(filtered_sentences)

[['The', 'Brandbergen', 'Mosque', 'officially', 'Islamic', 'Association', 'Brandbergen', 'mosque', 'located', 'Brandbergen', 'Haninge', 'Municipality', 'south', 'Stockholm', 'Sweden'], ['In', 'Arabic', 'language', 'manual', 'carried', 'logo', 'address', 'Brandbergen', 'Mosque', 'spread', 'internet'], ['The', 'manual', 'described', 'construction', 'simple', 'chemical', 'weapons', 'including', 'build', 'chemical', 'munition', 'ordinary', 'artillery', 'round'], ['On', 'December', 'Swedish', 'citizen', 'Mohamed', 'Moumou', 'described', 'United', 'States', 'Department', 'Treasury', 'uncontested', 'leader', 'extremist', 'group', 'centered', 'around', 'Brandbergen', 'Mosque', 'Stockholm', 'put', 'United', 'Nations', 'Security', 'Council', 'Committee', 'list', 'foreign', 'terrorists'], ['In', 'November', 'Islamic', 'Association', 'got', 'permission', 'build', 'Muslim', 'Cultural', 'Center', 'next', 'city', 'center'], ['The', 'cultural', 'center', 'include', 'assembly', 'hall', 'library', 'rest

In [27]:
model = Word2Vec(
    size=70,
    window=5,
    min_count=2,
    workers=multiprocessing.cpu_count(),
    iter=50
)

In [28]:
model.build_vocab(filtered_sentences)

In [29]:
model.train(
    filtered_sentences, 
    total_examples=model.corpus_count,
    epochs=model.epochs,
)

(193683, 519400)

In [30]:
model.wv.most_similar("terrorists")

[('foreign', 0.9925870895385742),
 ('list', 0.9537476301193237),
 ('Committee', 0.8970786929130554),
 ('Council', 0.8460586667060852),
 ('Security', 0.7554590106010437),
 ('Nations', 0.688389241695404),
 ('put', 0.5848478078842163),
 ('Stockholm', 0.4826817214488983),
 ('around', 0.4525851905345917),
 ('Mosque', 0.4181166887283325)]

In [31]:
model.wv.most_similar("artillery")

[('round', 0.9886975288391113),
 ('ordinary', 0.9842263460159302),
 ('munition', 0.9815595746040344),
 ('including', 0.9649131894111633),
 ('chemical', 0.9411182403564453),
 ('weapons', 0.91733717918396),
 ('simple', 0.8871848583221436),
 ('construction', 0.8371560573577881),
 ('build', 0.7539249658584595),
 ('Muslim', 0.5020869970321655)]

In [32]:
def analogy(start1, end1, end2):
    similarities = model.wv.most_similar(
       positive = [end2, start1],
       negative = [end1]
    )
    start2 = similarities[0][0]
    return start2

In [33]:
analogy("Sweden", "Swedish", "Arabic")

'located'

In [34]:
print(model.wv.doesnt_match("artillery munition weapons citizen".split()))

citizen


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
